In [1]:
from collections import Counter
from konlpy.tag import Hannanum
import pandas as pd
import pymongo
from pymongo import MongoClient
import urllib.request
import json
from konlpy.tag import Komoran
from konlpy.tag import Kkma

In [2]:
def get_hannanum_data(str):
    h = Hannanum()  # kkma = Kkma() , k = Komoran() , h = Hannanum()
    nouns = h.nouns(str)
    nouns_cut = []
    for i in nouns:
        if(len(i)>1):
            nouns_cut.append(i)

    count = Counter(nouns_cut)

    return count.most_common(50)

In [3]:
def get_mostCommon_result(pid,startDate,endDate):

    '''
    file_name = 'data/livetalk_%s.csv' % pid
    data = get_csv_text(file_name)
    text = data['BLTTHG_CNTNT']
    dic = []
    for i in text:
        dic.append(i)
    '''
    #startDate = '20160523'
    #endDate = '20160530'

    # http://10.52.37.26/text/liveTalk_ex.gs?itemCd=353813&startDate=20160523&endDate=20160530
    # http://10.52.37.26/text/estimate.gs?itemCd=339714&startDate=20160523&endDate=20160530
    file_path = 'http://10.52.37.26/text/liveTalk_ex.gs?itemCd=%s&startDate=%s&endDate=%s' % (pid,startDate,endDate)
    wp = urllib.request.urlopen(file_path)
    data_string = wp.read().decode('euc-kr')
    dfs = json.loads(data_string)

    dic = []
    for i in range(len(dfs)):
        dic.append(dfs[i]['cntnt'])

    str = ' '.join(dic)
    return get_hannanum_data(str)

In [4]:
get_mostCommon_result('338856','20160520','20160530')

[('아이오페', 29),
 ('구성', 29),
 ('구매', 19),
 ('주문', 17),
 ('쿠션', 13),
 ('방송', 10),
 ('대박', 9),
 ('제품', 9),
 ('엄마', 9),
 ('에어쿠션', 9),
 ('선물', 9),
 ('피부', 8),
 ('진짜', 6),
 ('여름', 6),
 ('사용', 6),
 ('화장품', 6),
 ('오늘', 5),
 ('주문완료', 5),
 ('스템셀', 4),
 ('완전', 4),
 ('민트케이스', 4),
 ('건조', 4),
 ('동생', 4),
 ('최고', 4),
 ('주름', 4),
 ('로션', 4),
 ('상상초월', 4),
 ('구입', 4),
 ('이유', 4),
 ('스킨', 3),
 ('ㅎㅎ', 3),
 ('소리', 3),
 ('가격', 3),
 ('사야', 3),
 ('ㅋㅋ', 3),
 ('만족', 3),
 ('언니', 3),
 ('알림신청하', 3),
 ('사은품', 3),
 ('커버력', 3),
 ('친정엄마', 3),
 ('혜택', 2),
 ('아이오페♡', 2),
 ('결제', 2),
 ('쿨러백', 2),
 ('기초', 2),
 ('이번', 2),
 ('지금', 2),
 ('지난번', 2),
 ('더운여름', 2)]

In [6]:
get_mostCommon_result('339714','20160520','20160530')

[('실크테라피', 57),
 ('구매', 46),
 ('주문', 25),
 ('머릿결', 20),
 ('사용', 16),
 ('방송', 15),
 ('구성', 14),
 ('가격', 13),
 ('머리', 11),
 ('머리결', 10),
 ('주문완료', 9),
 ('오늘', 9),
 ('구입', 9),
 ('제품', 8),
 ('최고', 7),
 ('ㅎㅎ', 5),
 ('이번', 5),
 ('필수품', 5),
 ('구매완료', 5),
 ('대박', 4),
 ('필요', 4),
 ('신랑', 4),
 ('선물', 4),
 ('고민', 4),
 ('남편', 3),
 ('믿고사', 3),
 ('손상', 3),
 ('윤기', 3),
 ('신청', 3),
 ('쟁여두', 3),
 ('저렴', 3),
 ('가격인상', 3),
 ('마음', 3),
 ('온가족', 3),
 ('가족', 3),
 ('한번', 3),
 ('미용실', 3),
 ('강추', 3),
 ('홈쇼핑', 3),
 ('쓰던거', 3),
 ('헤어에센스', 3),
 ('이거', 3),
 ('실크테라피~', 3),
 ('너무좋아요', 2),
 ('ㅋ이거', 2),
 ('ㅠㅠ', 2),
 ('실크테라피예요', 2),
 ('2세트주문완료요', 2),
 ('우리', 2),
 ('지금', 2)]

In [7]:
def get_db_connection():
    # connect to database
    connection = MongoClient('localhost',27017)
    db = connection.local
    return db

In [8]:
def insert_db(pid,ds,db):
    #db = db_connection()
    db.wordcnt.insert_one(
    {
        "item_code": pid,"wordlist":ds
    }
    )

In [21]:
db = get_db_connection()
# ds = get_mostCommon_result('339714','20160520','20160530')
insert_db('339714',get_mostCommon_result('339714','20160520','20160530'),db)

In [22]:
insert_db('338856',get_mostCommon_result('338856','20160520','20160530'),db)

In [23]:
insert_db('353813',get_mostCommon_result('353813','20160520','20160530'),db)

In [24]:
insert_db('441799',get_mostCommon_result('441799','20160520','20160530'),db)

In [13]:
def select_data(pid,db):
    #db = db_connection()
    dic = {}
    cursor = db.wordcnt.find({"item_code":pid})
    for document in cursor:
        dic = document

    return dic.get('wordlist')

def select_data_count(pid,db):
    #db = db_connection()
    dic = []
    cursor = db.wordcnt.find({"item_code":pid})
    for document in cursor:
        dic = document

    return len(dic)

In [14]:
select_data('441799',db)

[['실크테라피', 57],
 ['구매', 46],
 ['주문', 25],
 ['머릿결', 20],
 ['사용', 16],
 ['방송', 15],
 ['구성', 14],
 ['가격', 13],
 ['머리', 11],
 ['머리결', 10],
 ['주문완료', 9],
 ['오늘', 9],
 ['구입', 9],
 ['제품', 8],
 ['최고', 7],
 ['ㅎㅎ', 5],
 ['이번', 5],
 ['필수품', 5],
 ['구매완료', 5],
 ['대박', 4],
 ['필요', 4],
 ['신랑', 4],
 ['선물', 4],
 ['고민', 4],
 ['남편', 3],
 ['믿고사', 3],
 ['손상', 3],
 ['윤기', 3],
 ['신청', 3],
 ['쟁여두', 3],
 ['저렴', 3],
 ['가격인상', 3],
 ['마음', 3],
 ['온가족', 3],
 ['가족', 3],
 ['한번', 3],
 ['미용실', 3],
 ['강추', 3],
 ['홈쇼핑', 3],
 ['쓰던거', 3],
 ['헤어에센스', 3],
 ['이거', 3],
 ['실크테라피~', 3],
 ['너무좋아요', 2],
 ['ㅋ이거', 2],
 ['ㅠㅠ', 2],
 ['실크테라피예요', 2],
 ['2세트주문완료요', 2],
 ['우리', 2],
 ['지금', 2]]

In [16]:
def delete_all(pid,db):
    db.wordcnt.delete_many({"item_code":pid})

In [17]:
delete_all('339714',db)

In [18]:
delete_all('338856',db)

In [19]:
delete_all('353813',db)

In [20]:
delete_all('441799',db)